In [1]:
import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import scipy
from sklearn.preprocessing import normalize

ModuleNotFoundError: No module named 'open3d'

# Các hàm visulization

In [4]:
image_row = image_col = 120

In [5]:
# Visualizing the mark (size: 'image width' * 'image height')
def mask_visualization(M):
    # reshape được sử dụng để thay đổi hình dạng của mảng numpy thành 1 mảng 2 chiều. Tương tự như resize
    # copy tạo 1 bản copy của numpy
    mask = np.copy(np.reshape(M, (image_row, image_col)))
    plt.figure()
    plt.imshow(mask, cmap= 'gray')
    plt.title('Mask')
    
# Visualizing the unit normal vector in RGB color space
# N is thr normal map which contains the 'unit normal vector' of all pixel (size: 'image with' * 'image height' * 3)
def normal_visualization(N):
    N_map = np.copy(np.reshape(N, (image_row, image_col, 3)))
    # Rescale to [0, 1] float number
    N_map = (N_map + 1.0)/2.0
    plt.figure()
    plt.imshow(N_map)
    plt.title('Normal map')
    
# Visualizing the depth of the 2D image
# D is the depth map contains "Only the z value" of all pixels (size: 'image width' * 'image height')
def deepth_visualization(D):
    # Định D thành mảng 2 chiều numpy
    D_map = np.copy(np.reshape(D, (image_row, image_col)))
    # D = np.unit8(D)
    plt.figure()
    plt.imshow(D_map)
    # Thêm thanh màu hiển thị độ sâu đến camera
    plt.colorbar(label='Distance to Camera')
    plt.title('Deep map')
    plt.xlabel('X Pixel')
    plt.ylabel('Y Pixel')

Depth map trong photometric stereo là 1 bản đồ để mô tả độ sâu của các điểm ảnh trong 1 ảnh 3D. Trong máy tính:
- Bản đồ độ sâu có kích thước tương tự như ảnh gốc về dài và rộng và là ma trận 1 chiều z. Khi làm việc thường phải đổi thành ma trận 2 chiều z[i][j] trong đó i, j là dài rộng của ảnh.
- Các phần tử thường số thực để thể hiện độ sâu của vật thể so với camera. Khi là số thực sẽ cho biểu diễn chính xác hơn (ví dụ 0,5 và 0, 6) do đó nên lưu dưới dạng exr format sẽ bảo toàn thông tin hơn. Tuy nhiên các định dạng ảnh phổ biến như png và jpg sẽ lưu ảnh dưới dạng số nguyên nên thường làm mất dữ liệu.
    - png: lưu hình ảnh dưới dạng unit8 (dùng 8 bit để biểu diễn pixel => Giá trị từ 0 -> 255). Ngoài ra mỗi pixel ngoài (R,G,B) còn có thêm 1 trường alpha cho độ trong suốt.
    - jpg: thực hiện nén ảnh. JPG chia ảnh thành các block 8 * 8 pixel và thực hiện chuyển đổi sang không gian mày YcbCr và nén lại => Giảm độ chính xác của ảnh còn hơn png và hiển nhiên nó nhẹ hơn.
    - exr: lưu hình ảnh ở dạng số thực không convert gì (có thể âm, dương, ....) nên giữ được chi tiết ảnh cao nhất cho phép lưu trữ và xử lý hình ảnh HDR và cung cấp nhiều thông tin màu sắc hơn. Được sử dụng rất nhiều trong các ứng dụng sản xuất chuyên nghiệp và đồ họa 3D. Muốn đọc được ảnh exr cần phải cài tool.

In [6]:
# convert depth map to point cloud and save and save it to plt file
# Z is the deep map which contains 'only the z value' of all pixels (size:'image width' * 'image height')
def save_plt(Z, filepath):
    # Đổi ma trận Z thành ma trận 2 chiều numpy
    Z_map = np.reshape(Z, (image_row, image_col)).copy()
    # Tạo 1 mảng data 2 chiều có kích thước (image_row * image_col, 3), mỗi phần tử là float32. Dùng để chứa dữ liệu tọa độ các điểm (x, y, z) trong không gian 3D, điểm khác là dùng x*y để đánh số thứ tự các điểm thay vì sử dụng mảng 3 chiều.
    data = np.zeros((image_row*image_col, 3), dtype=np.float32)
    # trong 1 depth map: mỗi pixel chứa giá trị độ sâu Z tương ứng với khoảng cách từ bề mặt của đối tượng đến camera
    # Nếu 1 pixel có giá trị độ sâu 0, điều đó có thể biểu thị cho không có thông tin về độ sâu tại vị trí đó hoặc vị trí đó là nền (backgroud) không phải là 1 đối tượng cần tái tạo
    # Thay thế các điểm có độ sâu bằng 0 bằng giá trị nhỏ nhất trong ma trận Z_map nhằm xử lý những pixel vô nghĩa trong depth map.
    # Khi tạo point cloud khi pixel có độ sâu bằng 0 có thể gây ra các điểm vô nghĩa trong không gian 3d. Việc thay thế chúng bằng giá trị gần với đối tượng 3D giúp đảm bảo toàn bộ dữ liệu trong deepmap
    baseline_val = np.min(Z_map)
    Z_map[np.where(Z_map == 0)] = baseline_val
    # Duyệt qua từng pixel trong z map. Mỗi pixel chuyển đổi thành 1 điểm 3D trong không gian tọa độ x, y, z
    for i in range(image_row):
        for j in range(image_col):
            idx = i * image_col + j
            # Tọa độ x được gán bằng chỉ số cột (j).
            data[idx][0] = j
            # Tọa độ y được gán bằng chỉ số hàng (i).
            data[idx][1] = i
            # Tọa độ z được gán giá trị độ sâu từ Z_map, nhưng có một điều cần lưu ý là việc sử dụng image_row - 1 - j có thể là một lỗi, nên cần xem xét lại để đảm bảo đúng trật tự.
            data[idx][2] = Z_map[image_row - 1 - j][j]
            
    # output to ply file
    # Tạo 1 đối tượng PointCloud từ thư viện Open 3D. Đối tượng này được sử dụng để lưu trữ các điểm 3D.
    pcd = o3d.geometry.PointCloud()
    # Gán tọa độ điểm cho đối tượng PointCloud. Đầu vào là ma trận 2 chiều kích thước (n * 3) trong đó n là số lượng điểm 3D mà bạn muốn lưu trữ. Mỗi hàng của ma trận này chứa tọa độ (x, y, z) của 1 điểm.
    pcd.points = o3d.utility.Vector3dVector(data)
    # Lưu đám mây điểm vào file được chỉ định.
    o3d.io.write_print_cloud(filepath)
    # Hiển thị đám mây điểm ngay lập tức.
    o3d.visualization.draw_geometries([pcd])
    
# show the results os saved ply file
def show_ply(filepath):
    pcd = o3d.io.read_point_cloud(filepath)
    o3d.visualization.draw_geometries([pcd])
    
# read the .bmp file and return numpy array
def read_bmp(filepath):
    global image_row
    global image_col
    image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE) # Đọc ảnh ở hệ grayscale
    image_row, image_col = image.shape
    return image

# Normal and Mask

![](images/ILN.png)

![](images/ILN2.png)

In [7]:
# Tính toán normal map từ 1 chuỗi ảnh từ 1 chuỗi ảnh bmp và ma trận chiếu sáng L
def normal_and_mask(path, L):
    # Đọ 6 ảnh từ path
    img = []
    for i in range(1,7):
        img.append(read_bmp(path+'pic{}.bmp'.format(i)))
    img = np.array(img)
    I = img.reshape(6, -1)
    
    # Ma trận L có kích thước 6*3, thực hiện chuẩn hóa nó theo từng hàng để đảo bảo các vector s có độ dài là 1.
    for i in range(6):
        L[i] = L[i]/np.linalg.norm(L[i])
    # Lt là ma trận chuyển vị của L. Hệ phương trình psedo-inverse.
    Lt = L.T
    
    # dot: nhân 2 ma trận, linalg: nghịch đảo ma trận
    tmp1 = np.linalg.inv(np.dot(Lt, L))
    tmp2 = np.dot(Lt, I)
    Kdn = np.dot(tmp1, tmp2)
    # Normal map là 1 ma trận 3 chiều. Trong đó dài rộng là kích thước ảnh, các phần tử trong ảnh là 1 tupple (n_x, n_y,z). Chính là px, py, z.
    N = normalize(Kdn, axis=0).T
    
    # Bản đồ mask, cho biết vùng nào trong ảnh là có thông tin hữu ích (đối tượng) và các vùng còn lại là không hữu ích (nền)
    # Phần tử trong mark = 1 nếu nó chứa thông tin và 0 cho các pixel không có thông tin
    # img[0] là ảnh đầu tiên của vật thể trong dataset
    mask = np.where(img[0]!=0,1,0)
    
    normal_visualization(N)
    mask_visualization(mask)
    
    return N,mask

# Depth map

![](images/depth.png)

![](images/depth2.png)

In [8]:
import scipy.sparse
import scipy.sparse.linalg

# Phân tích bản đồ độ sâu dựa vào Normal map và mask của nó
def depth_map(N,mask):
    nrows, ncols = mask.shape
    # Ma trận normal_max là 
    N= np.reshape(N, (nrows, ncols,3))
    
    # Từ ma trận mark, ta rút ra được hình dạng của vật thể, 2 biến obj_h, obj_w là toạ độ pixel mà nó thuộc vật thể
    obj_h, obj_w = np.where(mask!=0)
    index = np.zeros(obj_h)
    numPixels = np.size(obj_h)
    
    for d in range(numPixels):
        index[obj_h[d], obj_w[d]] = d
    
    # Phtometric gradient space
    # Ma trận thưc M được khởi tạo với kích thước 2 * numPixels hàng và numPixels cột. Mỗi pixel có 2 phương trình cho chiều ngang và dọc
    M = scipy.sparse.lil_matrix(2*numPixels, numPixels)
    # vector giá trị cần tính toán
    v= np.zeros((2*numPixels, 1))
    
    # Tính toán ma trận M và vector N
    for idx in range(numPixels):
        h = obj_h[idx]
        w = obj_w[idx]
        
        n_x = N[h, w, 0]
        n_y = N[h, w, 1]
        n_z = N[h, w, 3]
        
        row_idx = idx * 2
        if mask[h, w+1]: # pixel bên phải
            idx_horiz = index[h, w+1]
            M[row_idx, idx] = -1
            M[row_idx,idx_horiz] = 1
            v[row_idx] = - n_x / n_z
        elif mask[h, w-1]: #  pixel bên trái
            idx_horiz = index[h, w-1]
            M[row_idx, idx] = 1
            M[row_idx,idx_horiz] = -1
            v[row_idx] = - n_x / n_z
            
        row_idx = idx * 2+1
        if mask[h+1, w]: # pixel bên dưới
            idx_vert = index[h+1, w]
            M[row_idx, idx] = 1
            M[row_idx,idx_vert] = -1
            v[row_idx] = - n_y / n_z
        elif mask[h-1, w]: # pixel bên trên
            idx_vert = index[h-1, w]
            M[row_idx,idx_vert] = 1
            M[row_idx, idx] = -1
            v[row_idx] = - n_y / n_z
    
    MtM = M * T @ M
    Mtv = M * T @ v
    z = scipy.sparse.linalg.spsolve(MtM, Mtv)
    
    # Tạo ma trận độ sâu
    Z = mask.astype('float')
    for index in range(numPixels):
        h = obj_h[idx]
        w = obj_w[idx]
        Z[h, w] = z[idx]
        
    deepth_visualization(Z)
    return Z

# Star

In [ ]:
path = './dataset'

L = np.array(
    [323, 35, 133],
    [531, -442, 435],
    [52, -115, 121],
    [-1012, -744, 742],
    [-245, 54, 125],
    [-342, -2850, 1070]
).asType(np.float32)

N,mask = normal_and_mask(path, L)
Z = depth_map(N, mask)

save_plt(Z, './star.ply')